In [10]:
!pip install pandas pandas_market_calendars

In [11]:
import pandas as pd
from pandas_market_calendars import get_calendar

## Reading data

In [12]:
equity_raw = pd.read_excel('Equity.xlsx', usecols=['DATE', 'MKT', 'SMB', 'HML',
                                                   'RMW', 'CMA', 'MOM', 'ST_REV', 'LT_REV'])

equity_raw.rename(columns=lambda x: 'ff_' + x if x != 'DATE' else x, inplace=True)
currency_raw = pd.read_excel('Currency.xlsx', usecols=['DATE', 'CAR',	'MOM1',	'MOM3',
                                                       'MOM6',	'MOM12',	'VOL',	'VRP'])

currency_raw.rename(columns=lambda x: 'fx_' + x if x != 'DATE' else x, inplace=True)
crypto_raw = pd.read_excel('Crypto.xlsx', usecols=['DATE', 'mkt_vw', 'twitter_followers_f',
                                                   'marketcap_f', 'bm_f', 'max30_f',
                                                   'r30_0_f', 'rvol30_f'])

crypto_raw.rename(columns=lambda x: 'cr_' + x if x != 'DATE' else x, inplace=True)

## Transforming column types

In [13]:
equity_raw['DATE'] = pd.to_datetime(equity_raw['DATE'], dayfirst=True)
currency_raw['DATE'] = pd.to_datetime(currency_raw['DATE'], dayfirst=True)
crypto_raw['DATE'] = pd.to_datetime(crypto_raw['DATE'], dayfirst=True)

## Merging data

In [14]:
equity_and_currency = pd.merge(equity_raw, currency_raw, on='DATE', how='inner')
merged_data = pd.merge(equity_and_currency, crypto_raw, on='DATE', how='inner')
merged_data = merged_data.sort_values(by='DATE')

In [15]:
print(merged_data.isnull().sum().sum())

0


In [16]:
columns = list(merged_data.columns)
columns.insert(2, columns.pop(columns.index('cr_mkt_vw')))
merged_data = merged_data[columns]
merged_data['fx_VOL'] = merged_data['fx_VOL'].apply(lambda x: x*(-1))
merged_data['cr_marketcap_f'] = merged_data['cr_marketcap_f'].apply(lambda x: x*(-1))
merged_data['cr_max30_f'] = merged_data['cr_max30_f'].apply(lambda x: x*(-1))
merged_data['cr_r30_0_f'] = merged_data['cr_r30_0_f'].apply(lambda x: x*(-1))
merged_data['cr_rvol30_f'] = merged_data['cr_rvol30_f'].apply(lambda x: x*(-1))
merged_data

,DATE,ff_MKT,cr_mkt_vw,ff_SMB,ff_HML,ff_RMW,ff_CMA,ff_MOM,ff_ST_REV,ff_LT_REV,...,fx_MOM6,fx_MOM12,fx_VOL,fx_VRP,cr_twitter_followers_f,cr_marketcap_f,cr_bm_f,cr_max30_f,cr_r30_0_f,cr_rvol30_f
0,2017-08-03,-0.0021,0.029858,-0.0035,-0.0033,0.0028,-0.0034,0.0029,-0.0013,-0.0046,...,-0.005930,-0.003534,0.004362,0.004376,0.095642,0.045064,0.039210,-0.001190,-0.013179,0.013891
1,2017-08-04,0.0025,-0.038056,0.0037,0.0039,-0.0003,0.0009,0.0002,0.0022,0.0016,...,0.005267,0.007898,0.003008,0.003819,0.062418,0.072311,-0.063277,-0.006528,0.032266,0.010590
2,2017-08-07,0.0016,0.052119,-0.0010,-0.0058,0.0024,-0.0036,0.0041,-0.0034,-0.0029,...,0.000750,0.001075,-0.005587,-0.003457,-0.023991,0.017440,0.031670,-0.097809,-0.071404,-0.056680
3,2017-08-08,-0.0024,0.037605,-0.0008,0.0024,-0.0033,-0.0019,0.0057,-0.0002,-0.0002,...,0.001284,0.002496,0.006532,0.006287,-0.030576,-0.003850,0.006963,0.050164,-0.016228,-0.025085
4,2017-08-09,-0.0014,-0.023488,-0.0080,-0.0007,0.0014,0.0004,-0.0022,-0.0019,-0.0029,...,0.002211,0.004370,0.003774,-0.001679,-0.020191,-0.030171,-0.016297,0.076809,0.137113,0.064925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,2023-01-25,0.0000,0.024653,0.0013,0.0065,0.0002,0.0022,0.0013,-0.0041,0.0058,...,0.000337,-0.002340,-0.003769,0.001710,-0.020903,0.007189,-0.005005,-0.068248,-0.048012,-0.079552
1379,2023-01-26,0.0108,-0.001309,-0.0062,0.0001,-0.0019,-0.0033,-0.0121,-0.0115,-0.0071,...,0.001659,0.000466,0.000715,-0.001797,0.000431,0.055110,-0.035475,0.035756,-0.011428,0.037430
1380,2023-01-27,0.0036,0.002978,0.0032,-0.0116,-0.0061,-0.0118,-0.0245,-0.0169,-0.0056,...,0.005871,0.005321,0.004840,0.000484,-0.009730,0.006671,-0.000433,-0.024882,-0.000199,-0.023274
1381,2023-01-30,-0.0138,-0.043258,0.0005,0.0072,0.0040,0.0081,0.0137,0.0185,0.0042,...,0.000494,0.002295,0.005881,0.005291,0.018118,-0.006466,0.103347,0.042973,0.048317,0.048398


## Checking the number of trading days

In [17]:
nyse = get_calendar("XNYS")
start_date = "2017-08-03"
end_date = "2023-01-31"
trading_days = nyse.valid_days(start_date=start_date, end_date=end_date)
num_trading_days = len(trading_days)

print(f"Number of trading days between {start_date} and {end_date}: {num_trading_days}")

Number of trading days between 2017-08-03 and 2023-01-31: 1383


Number of trading days corresponds to the number of time periods (i.e. rows) in dataset. This is an indication that the data has been merged correctly.

## Loading data

In [18]:
merged_data.to_csv('Data.csv', index=False)